# Makine Öğrenmesi Uygulanması

## Kütüphanelerin Eklenmesi
## Algoritmanın Uygulanması

In [24]:
# Kütüphanemizi ekliyoruz

In [30]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn import model_selection


#### son halini almış temiz verisetini okuyoruz.

In [2]:
df_2 = pd.read_excel(r"kopya.xlsx")
df_2.drop("Unnamed: 0", axis = 1, inplace = True)
df = df_2.copy()

In [3]:
df.columns

Index(['ilçe', 'türü', 'kategori', 'yapı_durumu', 'net_metrekare', 'alan',
       'oda_sayısı', 'bina_yaşı', 'dairenin_katı', 'kat_sayısı', 'ısıtma',
       'aidat', 'banyo_sayısı', 'wc_sayısı', 'site_içerisinde', 'eşya_durumu',
       'kullanım_durumu', 'krediye_uygunluk', 'yatırıma_uygunluk', 'yapı_tipi',
       'takas', 'fiyat'],
      dtype='object')

#### Çalışmak istediğimiz sütunları aldık.

In [4]:
df = df[["ilçe","türü","net_metrekare","alan","oda_sayısı","bina_yaşı","ısıtma","site_içerisinde",
         "eşya_durumu","banyo_sayısı","wc_sayısı","dairenin_katı","fiyat"]]

In [5]:
df

,ilçe,türü,net_metrekare,alan,oda_sayısı,bina_yaşı,ısıtma,site_içerisinde,eşya_durumu,banyo_sayısı,wc_sayısı,dairenin_katı,fiyat
0,6,1,165,190,6,4,5,0,0,1,2,16,430000
1,6,1,85,95,2,0,5,1,0,1,1,5,239000
2,6,1,130,143,4,2,5,1,0,2,0,5,400000
3,6,1,92,105,2,0,5,1,0,1,1,19,159000
4,6,1,120,140,5,3,5,1,0,1,1,10,385000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,5,1,85,103,2,7,6,1,0,2,0,13,750000
815,5,4,25,41,0,5,6,1,1,1,0,10,360000
816,5,1,65,74,1,7,6,1,0,1,0,11,550000
817,5,1,85,103,2,7,6,1,0,2,0,19,700000


#### Bağımlı ve Bağımsız değişkenleri ayırdık

In [6]:
X = df.drop(["fiyat"], axis = 1)
y = df["fiyat"]

#### Eğitim ve Test verisinin ayrılması

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 144)

In [32]:
# GridSearchCV 

In [10]:
params = {"colsample_bytree":[0.4,0.5,0.6],
         "learning_rate":[0.01,0.02,0.09],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[100,200,500,2000]}

In [13]:
xgb = XGBRegressor()

In [14]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

#### En uygun parametreleri giriyoruz

In [40]:
xgb1 = XGBRegressor(colsample_bytree = 0.5, learning_rate = 0.09, max_depth = 4, n_estimators = 2000)

#### Modelimizi eğitiyoruz

In [41]:
model_xgb = xgb1.fit(X_train, y_train)

#### Tahmin yapıyoruz

In [42]:
model_xgb.predict(X_test)[15:20]

array([ 246285.84, 1144784.  ,  658234.56,  609021.2 ,  381962.12],
      dtype=float32)

#### tahmin edilen ve gerçek verileri karşılaştırabiliriz

In [43]:
y_test[15:20]

489     250000
431    1750000
279     635000
677    1350000
200     400000
Name: fiyat, dtype: int64

#### modelin skorunu hesapladık. 0-1 arası değer döner bize

In [44]:
model_xgb.score(X_test, y_test)

0.8500945754454254

In [45]:
model_xgb.score(X_train, y_train)

0.9999980594753378

#### valide edilmiş (doğrulanmış hata oranımızı buluyoruz)

In [46]:
np.sqrt(-1*(cross_val_score(model_xgb, X_test, y_test, cv=10, scoring='neg_mean_squared_error'))).mean()

5663891.709316989

#### model de parametrelerin önemini görüyoruz

In [47]:
importance = pd.DataFrame({"Importance": model_xgb.feature_importances_},
                         index=X_train.columns)

In [48]:
importance

,Importance
ilçe,0.032922
türü,0.334605
net_metrekare,0.059210
alan,0.062562
oda_sayısı,0.022739
bina_yaşı,0.044012
ısıtma,0.008799
site_içerisinde,0.000181
eşya_durumu,0.000624
banyo_sayısı,0.236852
